In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile2 = pd.read_pickle("DDD_Train_Decile2.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile2.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile2, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4763 - acc: 0.7743 - val_loss: 0.4748 - val_acc: 0.7743
Epoch 58/75
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4731 - acc: 0.7858 - val_loss: 0.4748 - val_acc: 0.7743
Epoch 59/75
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4743 - acc: 0.7867 - val_loss: 0.4747 - val_acc: 0.7707
Epoch 60/75
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4704 - acc: 0.7845 - val_loss: 0.4748 - val_acc: 0.7743
Epoch 61/75
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4739 - acc: 0.7783 - val_loss: 0.4748 - val_acc: 0.7743
Epoch 62/75
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4736 - acc: 0.7800 - val_loss: 0.4749 - val_acc: 0.7743
Epoch 63/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4750 - acc: 0.7761 - val_loss: 0.4748 - val_acc: 0.7743
Epoch 64/75
2264/2264 [==============================] - 4s 2ms/ste

Epoch 41/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4711 - acc: 0.7769 - val_loss: 0.4755 - val_acc: 0.7672
Epoch 42/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4829 - acc: 0.7747 - val_loss: 0.4756 - val_acc: 0.7672
Epoch 43/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4787 - acc: 0.7708 - val_loss: 0.4756 - val_acc: 0.7672
Epoch 44/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4764 - acc: 0.7796 - val_loss: 0.4755 - val_acc: 0.7672
Epoch 45/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4799 - acc: 0.7774 - val_loss: 0.4756 - val_acc: 0.7672
Epoch 46/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4745 - acc: 0.7747 - val_loss: 0.4756 - val_acc: 0.7672
Epoch 47/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4772 - acc: 0.7809 - val_loss: 0.4755 - val_acc: 0.7672
Epoch 48/100
2264/2264 [========================

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4765 - acc: 0.7800 - val_loss: 0.4782 - val_acc: 0.7707
Epoch 57/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4811 - acc: 0.7756 - val_loss: 0.4783 - val_acc: 0.7725
Epoch 58/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4722 - acc: 0.7836 - val_loss: 0.4782 - val_acc: 0.7707
Epoch 59/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4759 - acc: 0.7800 - val_loss: 0.4781 - val_acc: 0.7707
Epoch 60/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4777 - acc: 0.7858 - val_loss: 0.4781 - val_acc: 0.7707
Epoch 61/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4772 - acc: 0.7769 - val_loss: 0.4780 - val_acc: 0.7707
Epoch 62/75
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4746 - acc: 0.7814 - val_loss: 0.4781 - val_acc: 0.7707
Epoch 63/75
2264/2264 [==============================] - 4s 2ms/ste

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4728 - acc: 0.7814 - val_loss: 0.4735 - val_acc: 0.7725
Epoch 40/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4726 - acc: 0.7774 - val_loss: 0.4735 - val_acc: 0.7725
Epoch 41/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4710 - acc: 0.7800 - val_loss: 0.4735 - val_acc: 0.7725
Epoch 42/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4722 - acc: 0.7809 - val_loss: 0.4735 - val_acc: 0.7725
Epoch 43/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4709 - acc: 0.7756 - val_loss: 0.4734 - val_acc: 0.7725
Epoch 44/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4763 - acc: 0.7721 - val_loss: 0.4734 - val_acc: 0.7725
Epoch 45/125
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4733 - acc: 0.7699 - val_loss: 0.4734 - val_acc: 0.7725
Epoch 46/125
2264/2264 [==============================] - 3s 

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4827 - acc: 0.7752 - val_loss: 0.4733 - val_acc: 0.7707
Epoch 100/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4726 - acc: 0.7845 - val_loss: 0.4733 - val_acc: 0.7707
Epoch 101/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4717 - acc: 0.7761 - val_loss: 0.4732 - val_acc: 0.7707
Epoch 102/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4773 - acc: 0.7761 - val_loss: 0.4733 - val_acc: 0.7707
Epoch 103/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4732 - acc: 0.7699 - val_loss: 0.4733 - val_acc: 0.7707
Epoch 104/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4760 - acc: 0.7796 - val_loss: 0.4733 - val_acc: 0.7707
Epoch 105/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4785 - acc: 0.7792 - val_loss: 0.4732 - val_acc: 0.7707
Epoch 106/125
2264/2264 [==============================

2264/2264 [==============================] - 4s 2ms/step - loss: 0.5080 - acc: 0.7681 - val_loss: 0.4914 - val_acc: 0.7513
Epoch 34/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5062 - acc: 0.7752 - val_loss: 0.4912 - val_acc: 0.7513
Epoch 35/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5012 - acc: 0.7699 - val_loss: 0.4910 - val_acc: 0.7566
Epoch 36/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4976 - acc: 0.7672 - val_loss: 0.4909 - val_acc: 0.7549
Epoch 37/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5007 - acc: 0.7734 - val_loss: 0.4904 - val_acc: 0.7549
Epoch 38/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.5018 - acc: 0.7734 - val_loss: 0.4902 - val_acc: 0.7549
Epoch 39/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4951 - acc: 0.7747 - val_loss: 0.4900 - val_acc: 0.7531
Epoch 40/100
2264/2264 [==============================] - 4s 

Epoch 93/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4960 - acc: 0.7743 - val_loss: 0.4841 - val_acc: 0.7601
Epoch 94/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4832 - acc: 0.7796 - val_loss: 0.4840 - val_acc: 0.7601
Epoch 95/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4988 - acc: 0.7721 - val_loss: 0.4841 - val_acc: 0.7601
Epoch 96/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4921 - acc: 0.7778 - val_loss: 0.4841 - val_acc: 0.7601
Epoch 97/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4914 - acc: 0.7792 - val_loss: 0.4840 - val_acc: 0.7601
Epoch 98/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4945 - acc: 0.7739 - val_loss: 0.4841 - val_acc: 0.7584
Epoch 99/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4952 - acc: 0.7765 - val_loss: 0.4840 - val_acc: 0.7601
Epoch 100/100
2264/2264 [=======================

2264/2264 [==============================] - 3s 2ms/step - loss: 0.4918 - acc: 0.7730 - val_loss: 0.4944 - val_acc: 0.7584
Epoch 112/125
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4889 - acc: 0.7743 - val_loss: 0.4942 - val_acc: 0.7584
Epoch 113/125
2264/2264 [==============================] - 2s 822us/step - loss: 0.4805 - acc: 0.7800 - val_loss: 0.4799 - val_acc: 0.7690
Epoch 14/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4824 - acc: 0.7725 - val_loss: 0.4792 - val_acc: 0.7690
Epoch 15/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4793 - acc: 0.7831 - val_loss: 0.4787 - val_acc: 0.7672
Epoch 16/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4858 - acc: 0.7716 - val_loss: 0.4774 - val_acc: 0.7725
Epoch 17/50
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4845 - acc: 0.7774 - val_loss: 0.4779 - val_acc: 0.7707
Epoch 18/50
2264/2264 [==============================] - 3s 1

In [ ]:
0.86434